In [2]:
import pandas as pd
import urllib.request
import tarfile
import os
import random
import time
import threading
import datetime
import logging
import copy
from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
data = "train.csv"
images = pd.read_csv(data)

# enter the directory to save the images to.
directory = ""

# enter the directory of the training data. 
train_dir = ""

upper = 5
lower = 3
max_categories = 2000
num_machines = 5
random_state = 1234

In [4]:
def resize(sub, file, new_path):
    if os.path.isfile(os.path.join(sub, file)):
        try:
            img = Image.open(os.path.join(sub, file))
            img.verify()
            img = Image.open(os.path.join(sub, file))
            new_img = img.resize((800, 800))
        except:
            print("Failed: ", os.path.join(sub, file))
            return False
        
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        
        new_img.save(os.path.join(new_path, file))
        return True
    else:
        print("error opening file: {}\{}".format(sub, file))
        return False

In [5]:
def download(url, path, train_path, filename): 
    if not os.path.exists(path):
        os.makedirs(path)
    destination = "{}\{}".format(path, filename)
    train_location = '{}\{}'.format(train_path, filename)
    
    # if the image already exists in the training data set, return false.
    if os.path.isfile(train_location):
        print("{} already exists".format(train_location))
        return False
    
    try: 
        urllib.request.urlretrieve(url, destination)
        resize(path, filename, path)
        print("success: ")
        return True
    except: 
    # if download failed, return false.
        print("failed: ")
        return False
    finally: 
        print("download {} to {}".format(url, destination))

In [6]:
def manage_download(selected, backup):
    backup_index = 0
    for index, row in selected.iterrows():
        url = row['url']
        path = "{}\{}".format(directory, row['landmark_id'])
        filename = "{}.jpg".format(row['id'])
        train_path = '{}\{}'.format(train_dir, row['landmark_id'])
        
        while not download(url, path, train_path, filename) and backup_index < len(backup):
            url = backup["url"].iloc[backup_index]
            path = "{}\{}".format(directory, backup["landmark_id"].iloc[backup_index])
            train_path = "{}\{}".format(train_dir, backup["landmark_id"].iloc[backup_index])
            filename = "{}.jpg".format(backup["id"].iloc[backup_index])
            backup_index += 1 

In [7]:
for c in os.listdir(train_dir):
    print("category: {}".format(c))
    target_size = random.randint(lower, upper)
    matched_images = images[images["landmark_id"] == int(c)]
    selected = matched_images.sample(target_size, random_state=random_state)
    backup = matched_images[~matched_images['id'].isin(selected['id'])]
    manage_download(selected, backup)
    clear_output(wait=True)

category: 99851
E:\BigDataNew\99851\5648641f420b60b7.jpg already exists
success: 
download https://upload.wikimedia.org/wikipedia/commons/2/24/Die_Geislerspitzen_und_Sankt_Magdalena.jpg to E:\TestData\99851\53a8374a2fb0621f.jpg
success: 
download https://upload.wikimedia.org/wikipedia/commons/f/fa/Ncisles_udu_dal_Col_Raiser_Sas_dla_Porta_Cansles_Duleda_Gherd%C3%ABina.jpg to E:\TestData\99851\c629eccc280f326a.jpg
E:\BigDataNew\99851\faf0d70ac1a25cd4.jpg already exists
success: 
download http://upload.wikimedia.org/wikipedia/commons/b/b4/Odles_ududes_da_Funes_5.JPG to E:\TestData\99851\9e345696d94026bb.jpg
